In [3]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [4]:
mnist = tf.keras.datasets.mnist
# Load its data into training and test vectors
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Collect feature size info
imgSize0=len(x_train[0])
imgSize1=len(x_train[0][0])
numPixels=imgSize0*imgSize1
numTrainImages=len(x_train)
featureShape=(None, imgSize0,imgSize1,1)

# Clearup everything before running
tf.keras.backend.clear_session()
# Create model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)
    ),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu',),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])
# Build model and print summary
model.build(input_shape=featureShape)
model.summary()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                8010      
Total params: 17,578
Trainable params: 17,578
Non-trainable params: 0
____________________________________________________

In [5]:
history = model.fit(x_train, y_train, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1882 - accuracy: 0.9440
Epoch 2/2
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0633 - accuracy: 0.9806


In [6]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)

def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_train.astype('float32')).batch(1).take(100):
      yield [input_value]   
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization (except inputs/outputs which are always float)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
model_tflite = converter.convert()

# tflite_model = converter.convert()
model_length = open("model.tflite", "wb").write(model_tflite)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /var/folders/mv/1txqzr191f1b8gyq0sjcwyth0000gn/T/tmph8zo39eg/assets


In [7]:
print(model_length)

23632


In [8]:
!xxd -i 'model.tflite' > 'model.cc'

In [9]:
#  Read in the file
with open('model.cc', 'r') as file :
  filedata = file.read()

#include "model.h"
# Replace the target string
filedata = filedata.replace('unsigned char', 'alignas(16) const unsigned char' )
filedata = filedata.replace('unsigned int', 'const int' )
filedata = '#include "model.h" \n \n' \
    + "// Keep model aligned to 8 bytes to guarantee aligned 64-bit accesses. \n\n" \
    + filedata
# Write the file out again
with open('model.cc', 'w') as file:
  file.write(filedata)

In [10]:
read_h_file = open('model.h', 'r') 
old_lines = read_h_file.readlines()
new_lines = []    
for line in old_lines:
    if "model_len" in line:
        new_lines.append("const int model_len = {};".format(model_length) )
    else:
        new_lines.append(line)
# print (new_lines)
read_h_file.close()

write_header_file = open('model.h', 'w') 
write_header_file.writelines(new_lines)
write_header_file.close()

In [11]:
# print(new_lines)

['/* Copyright 2020 The TensorFlow Authors. All Rights Reserved.\n', '\n', 'Licensed under the Apache License, Version 2.0 (the "License");\n', 'you may not use this file except in compliance with the License.\n', 'You may obtain a copy of the License at\n', '\n', '    http://www.apache.org/licenses/LICENSE-2.0\n', '\n', 'Unless required by applicable law or agreed to in writing, software\n', 'distributed under the License is distributed on an "AS IS" BASIS,\n', 'WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n', 'See the License for the specific language governing permissions and\n', 'limitations under the License.\n', '==============================================================================*/\n', '\n', '// Automatically created from a TensorFlow Lite flatbuffer using the command:\n', '// xxd -i model.tflite > model.cc\n', '\n', '// This is a standard TensorFlow Lite model file that has been converted into a\n', '// C data array, so it can be easily com